Proposed steps
* Visit direct link at https://www.sb.com/ng/sport/football/live_list
* Identify HT tag ("game-id")
* Identify scores tags (two "score-item" within the "score" div)
* Identify matches with 0 total goals at HT (add the two "score-item")
* Extract [title] of class "teams" and extract home and away team names (the tags are "home-team" and "away-team")
* Extracted data should be loaded into a csv file with the column titles: title,home-team,away-team,ht_goals
* Read the output csv with pandas and display the content

### Import Required Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import json
import os
import random
from urllib.parse import urljoin
import csv
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# # SoccerData imports (install with: pip install soccerdata)
# try:
#     import soccerdata as sd
#     SOCCERDATA_AVAILABLE = True
#     print("✅ SoccerData library available")
# except ImportError:
#     SOCCERDATA_AVAILABLE = False
#     print("⚠️ SoccerData library not installed. Run: pip install soccerdata")

# Uncomment the following lines if packages are not installed
# print("📦 Installing required packages...")
# !pip install requests beautifulsoup4 pandas lxml selenium webdriver-manager

⚠️ SoccerData library not installed. Run: pip install soccerdata


### Load Utility Functions

In [ ]:
def get_random_headers():
    """Load and return a random set of headers from the JSON file."""
    # Get the directory where the current script is located
    script_dir = os.path.dirname(os.path.abspath(__file__))
    headers_file = os.path.join(script_dir, 'browser_headers.json')
    
    try:
        with open(headers_file, 'r') as f:
            headers_list = json.load(f)
        
        # Return a random set of headers
        return random.choice(headers_list)
    
    except FileNotFoundError:
        # Fallback to your original headers if file not found
        return {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }


def scrape_sb_live():
    """
    Scrapes sb live football matches and extracts halftime data
    Returns a list of dictionaries containing match data
    """
    url = "https://www.sportybet.com/ng/sport/football/live_list"
    
    # Headers to mimic a real browser
    # headers = get_random_headers()
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1"
    }
    
    print("🌐 Fetching data...")
    
    try:
        # Set up headless Chrome
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # Run without opening a browser window
        chrome_options.add_argument("--no-sandbox")  # For stability in some environments
        chrome_options.add_argument("--disable-dev-shm-usage")  # Avoid resource issues
        chrome_options.add_argument("--disable-gpu")  # Additional stability
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-logging")  # Reduce log noise
        chrome_options.add_argument("--log-level=3")  # Only fatal errors
        chrome_options.add_argument(f"user-agent={headers['User-Agent']}")  # Reuse your user-agent for consistency
        
        # Initialize driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        print("🛠️ Initializing browser...")
        driver.get(url)
        
        # Wait for JS to load (adjust timeout if needed; 10 seconds should suffice for this site)
        driver.implicitly_wait(10)
        
        # print("✅ Page loaded with JS rendered")

        # Get page source and clean it before parsing
        page_source = driver.page_source
        
        # Clean the page source to remove any problematic content
        # Remove any WebDriver-related paths that might be causing issues
        page_source = re.sub(r'/[^<>]*?\.wdm/[^<>]*?chromedriver[^<>]*?', '', page_source)
        page_source = re.sub(r'\[[^<>\[\]]*?chromedriver[^<>\[\]]*?\]', '', page_source)

        # Parse with explicit parser and error handling
        try:
            # Try html.parser first (most robust)
            soup = BeautifulSoup(page_source, 'html.parser')
        except Exception as e1:
            print(f"⚠️ html.parser failed: {e1}")
            try:
                # Fallback to lxml if available
                soup = BeautifulSoup(page_source, 'lxml')
            except Exception as e2:
                print(f"⚠️ lxml parser failed: {e2}")
                # Last resort - use html5lib if available
                try:
                    soup = BeautifulSoup(page_source, 'html5lib')
                except Exception as e3:
                    print(f"❌ All parsers failed. html5lib error: {e3}")
                    return []
        
        # # Parse the fully rendered HTML
        # soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # driver.quit()  # Clean up browser session
        
        # Find all matches with the correct class structure
        matches = soup.find_all('div', class_='m-table-row m-content-row match-row football-row')
        print(f"🎮 Found {len(matches)} ongoing events")
        
        extracted_data = []
        halftime_matches = 0
        first_half_matches = 0
        second_half_matches = 0
        zero_goal_matches = 0

        # # Load watchlist CSV
        # try:
        #     watchlist_df = pd.read_csv('watchlist_today.csv')
        #     watchlist_titles = set(watchlist_df['title'])  # Convert titles to a set for O(1) lookup
        # except Exception as e:
        #     print(f"⚠️ Could not load watchlist_today.csv: {e}")
        #     watchlist_titles = set()
        
        for match in matches:
            try:                
                # Check if this is a halftime match
                left_team_cell = match.find(class_='m-table-cell left-team-cell')
                is_halftime = False
                is_first_half = False
                is_second_half = False

                if left_team_cell:
                    left_team_table = left_team_cell.find(class_='left-team-table')
                    if left_team_table:
                        game_id_elem = left_team_table.find(class_='game-id')
                        if game_id_elem:
                            time_text = game_id_elem.get_text(strip=True).upper()
                            # print(f"Game ID text: {time_text}")  # Debug output
                            is_halftime = any(x in time_text for x in ['HT', 'HALF', 'HALFTIME', 'HALF-TIME'])
                            is_first_half = any(x in time_text for x in ['H1', '1ST', 'FIRST'])
                            is_second_half = any(x in time_text for x in ['H2', '2ND', 'SECOND'])

                # Skip if not a halftime, first half, or second half match
                if not (is_halftime or is_first_half or is_second_half):
                    continue

                # Update counters
                if is_halftime:
                    halftime_matches += 1
                if is_first_half:
                    first_half_matches = first_half_matches + 1 if 'first_half_matches' in locals() else 1
                if is_second_half:
                    second_half_matches = second_half_matches + 1 if 'second_half_matches' in locals() else 1
                
                # Find teams container
                teams_container = match.find(class_='teams')
                if not teams_container:
                    continue
                
                # Extract team names
                home_team_elem = teams_container.find(class_='home-team')
                away_team_elem = teams_container.find(class_='away-team')
                
                if not home_team_elem or not away_team_elem:
                    continue
                
                home_team = home_team_elem.get_text(strip=True)
                away_team = away_team_elem.get_text(strip=True)
                
                # Extract title from teams container
                title = teams_container.get('title', f"{home_team} vs {away_team}")
                
                # Find score container
                score_container = match.find(class_='score')
                if not score_container:
                    continue
                
                # Find score items
                score_items = score_container.find_all(class_='score-item')
                if len(score_items) < 2:
                    continue
                
                # Extract scores and convert to integers
                try:
                    home_score = int(score_items[0].get_text(strip=True))
                    away_score = int(score_items[1].get_text(strip=True))
                    total_goals = home_score + away_score
                except (ValueError, IndexError):
                    continue
                
                # Only include matches with 0 total goals at HT
                if total_goals == 0 and is_halftime:
                    match_data = {
                        'title': title,
                        'home-team': home_team,
                        'away-team': away_team,
                        'ht_goals': total_goals
                    }
                    extracted_data.append(match_data)
                    zero_goal_matches += 1
                    print(f"⚽ 0-goal HT event: {home_team} vs {away_team}")
                    # if title in watchlist_titles:
                    #     print(f"👀⭐ Watchlist event: {home_team} vs {away_team}")
                    # else:
                    #     print(f"⚽ 0-goal HT event: {home_team} vs {away_team}")
                    # Attempt to find SofaScore URL
                    # print(search_sofascore_match(home_team, away_team))
            
            except Exception as e:
                print(f"⚠️ Error processing match: {e}")
                continue
        
        print(f"\n📊 Summary:")
        # print(f"   - Total events found: {len(matches)}")
        print(f"   - HT: {halftime_matches}, H1: {first_half_matches}, H2: {second_half_matches}")
        print(f"   - 0 at HT events: {zero_goal_matches}")
        
        return extracted_data
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching data: {e}")
        return []
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return []


def save_to_csv(data, filename=None):
    """
    Save extracted data to CSV file with timestamp
    """
    if not data:
        print("❌ No data to save")
        return False, None
    
    if filename is None:
        # Generate filename with current timestamp
        current_time = datetime.now()
        filename = f"sb_default_interactive_{current_time.strftime('%d-%m-%y-%H-%M-%S')}.csv"
    
    try:
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"💾 Data saved to {filename}")
        return True, filename
    except Exception as e:
        print(f"❌ Error saving to CSV: {e}")
        return False, None
        

def generate_stats_prompts(extracted_data, output_file='stats_prompts.txt'):
    """
    Generates statistical analysis prompts for each match in extracted_data
    and saves them to a text file for easy copy/paste

    Args:
        extracted_data (list): List of dictionaries containing match data from scrape_sb_live()
        output_file (str): Name of the output text file (default: 'stats_prompts.txt')

    Returns:
        int: Number of prompts generated
    """

    # Check if extracted_data is empty
    if not extracted_data:
        print("📝 No matches found - no prompts to generate.")
        return 0

    # Get current date for the prompts (format: YYYY-MM-DD)
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Prompt template
    prompt_template = """Provide the following stats for the last 5 matches of both teams in the football match {home_team} vs {away_team} played on {date} (format: YYYY-MM-DD):
* Average number of goals scored by {home_team} in the last 5 matches
* Average number of goals conceded by {home_team} in the last 5 matches
* Total number of goals scored by {home_team} in the last 5 matches
* Total number of goals conceded by {home_team} in the last 5 matches
* Total season stats for {home_team} (games played, games won, games drawn, games lost, goals scored and conceded, if available)
* Average number of goals scored by {away_team} in the last 5 matches
* Average number of goals conceded by {away_team} in the last 5 matches
* Total number of goals scored by {away_team} in the last 5 matches
* Total number of goals conceded by {away_team} in the last 5 matches
* Total season stats for {away_team} (games played, games won, games drawn, games lost, goals scored and conceded, if available)
Provide live stats (shots on target, off target, corners) for {home_team} vs {away_team} on {date}. 
Calculate over 0.5 goals probability using Poisson model from last 5 match averages.
List only these stats for each team, in this order, with no additional text. Use 'N/A' if data is unavailable."""

    try:
        # Generate prompts for all matches
        all_prompts = []

        for i, match in enumerate(extracted_data, 1):
            home_team = match['home-team']
            away_team = match['away-team']

            # Generate the prompt for this match
            prompt = prompt_template.format(
                home_team=home_team,
                away_team=away_team,
                date=current_date
            )

            # Add separator and match info for clarity
            match_header = f"\n{'='*80}\nMATCH {i}: {home_team} vs {away_team} (0 goals at HT)\n{'='*80}\n"
            full_prompt = match_header + prompt

            all_prompts.append(full_prompt)

        # Write all prompts to file
        with open(output_file, 'w', encoding='utf-8') as f:
            # Add file header with timestamp
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            file_header = f"FOOTBALL STATS PROMPTS - Generated on {timestamp}\n"
            file_header += f"Total matches: {len(extracted_data)}\n"
            file_header += "="*80 + "\n"

            f.write(file_header)

            # Write all prompts
            for prompt in all_prompts:
                f.write(prompt)
                f.write("\n\n")  # Add spacing between prompts

            # Add footer
            footer = f"\n{'='*80}\nEND OF PROMPTS - {len(extracted_data)} matches total\n{'='*80}"
            f.write(footer)

        # print(f"📝 Generated {len(extracted_data)} stat prompts and saved to '{output_file}'")
        # print(f"💾 File size: {os.path.getsize(output_file)} bytes")

        return len(extracted_data)

    except Exception as e:
        print(f"❌ Error generating prompts: {e}")
        return 0



def display_results(filename=None):
    """
    Read and display the CSV file contents
    """
    try:
        if filename is None:
            print("❌ No filename provided")
            return None
            
        df = pd.read_csv(filename)
        print(f"\n📋 Contents of {filename}:")
        print("=" * 60)
        print(df.to_string(index=False))
        print(f"\n📈 Dataset Info:")
        print(f"   - Shape: {df.shape}")
        print(f"   - Columns: {list(df.columns)}")
        return df
    except FileNotFoundError:
        print(f"❌ File {filename} not found")
        return None
    except Exception as e:
        print(f"❌ Error reading CSV: {e}")
        return None



### Scrape fresh data

In [3]:
matches_data = scrape_sb_live()

🌐 Fetching data...
🛠️ Initializing browser...
🎮 Found 0 ongoing events

📊 Summary:
   - HT: 0, H1: 0, H2: 0
   - 0 at HT events: 0


In [ ]:
print("🚀 Starting sb Scraper")
print("=" * 50)

# Step 1: Scrape the website
matches_data = scrape_sb_live()

if matches_data:
    # Step 2: Save to CSV with timestamp
    success, csv_filename = save_to_csv(matches_data)
    if success and csv_filename:
        # Step 3: Read and display results
        df = display_results(csv_filename)
        
        if df is not None and not df.empty:
            print(f"\n🎉 Successfully processed {len(df)} matches with 0 HT goals!")
            print(f"📁 File saved as: {csv_filename}")
        else:
            print("\n⚠️ No matches found with 0 total goals at halftime")
else:
    print("\n❌ No data extracted. Please check the website structure or network connection.")

print("\n✅ Script execution completed!")

### Save to CSV with timestamp

In [8]:
success, filename = save_to_csv(matches_data)

# Display results using returned filename"
df = display_results(filename)

❌ No data to save
❌ No filename provided


### Extract stats

### Update csv with extracted stats

In [ ]:
# updated_file = update_csv_with_soccerdata(filename)
# final_df = display_results(updated_file)

In [ ]:
import time
import random
import re
import pandas as pd
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup


def get_random_headers():
    """Generate random headers to mimic real browser requests"""
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    ]
    
    return {
        'User-Agent': random.choice(user_agents),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
    }


def save_to_csv(data, filename=None):
    """
    Save extracted data to CSV file with timestamp
    """
    if not data:
        print("❌ No data to save")
        return False, None
    
    if filename is None:
        # Generate filename with current timestamp
        current_time = datetime.now()
        filename = f"sb_{current_time.strftime('%d-%m-%y-%H-%M-%S')}.csv"
    
    try:
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"💾 Data saved to {filename}")
        return True, filename
    except Exception as e:
        print(f"❌ Error saving to CSV: {e}")
        return False, None


def calculate_total_goals(score_text):
    """
    Extract and calculate total goals from score string like '0:2 ' or '1:4 '
    """
    try:
        # Clean the score text and extract numbers
        score_clean = score_text.strip().replace(' ', '')
        if ':' in score_clean:
            home_goals, away_goals = score_clean.split(':')
            return int(home_goals) + int(away_goals)
        return 0
    except (ValueError, AttributeError):
        return 0


def select_date(driver, target_date):
    """
    Select a specific date from the dropdown
    Args:
        driver: Selenium WebDriver instance
        target_date: Date string in format "05/09/2025"
    """
    try:
        print(f"🗓️ Attempting to select date: {target_date}")
        
        # Wait for the date dropdown to be present
        wait = WebDriverWait(driver, 20)
        
        # Try to find and click the dropdown
        dropdown_selectors = [
            ".m-select-list",
            ".optionEvent .m-select-list",
            "[class*='select-list']"
        ]
        
        dropdown_element = None
        for selector in dropdown_selectors:
            try:
                dropdown_element = wait.until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
                )
                print(f"✅ Found dropdown with selector: {selector}")
                break
            except TimeoutException:
                continue
        
        if not dropdown_element:
            print("❌ Could not find date dropdown")
            return False
        
        # Click the dropdown to open it
        driver.execute_script("arguments[0].click();", dropdown_element)
        time.sleep(2)
        
        # Look for date options
        date_options = driver.find_elements(By.CSS_SELECTOR, ".m-select-list span, .select-index")
        
        # If no options found, try alternative selectors
        if not date_options:
            date_options = driver.find_elements(By.XPATH, f"//*[contains(text(), '{target_date}')]")
        
        for option in date_options:
            try:
                option_text = option.text.strip()
                print(f"🔍 Found date option: '{option_text}'")
                
                if target_date in option_text:
                    print(f"✅ Selecting date: {target_date}")
                    driver.execute_script("arguments[0].click();", option)
                    time.sleep(3)  # Wait for page to reload with new date
                    return True
            except Exception as e:
                print(f"⚠️ Error checking date option: {e}")
                continue
        
        print(f"❌ Date '{target_date}' not found in available options")
        return False
        
    except Exception as e:
        print(f"❌ Error selecting date: {e}")
        return False


def check_and_navigate_pagination(driver):
    """
    Check if there are more pages and navigate to the next one
    Returns True if successfully navigated to next page, False if no more pages
    """
    try:
        # Look for next page button - try multiple selectors
        next_selectors = [
            ".pagination .pageNum.icon-next:not(.icon-disabled)",
            ".pagination .icon-next:not(.icon-disabled)",
            "[class*='icon-next']:not([class*='disabled'])"
        ]
        
        for selector in next_selectors:
            try:
                next_button = driver.find_element(By.CSS_SELECTOR, selector)
                if next_button and "icon-disabled" not in next_button.get_attribute("class"):
                    print("📄 Navigating to next page...")
                    driver.execute_script("arguments[0].click();", next_button)
                    time.sleep(3)  # Wait for page to load
                    return True
            except NoSuchElementException:
                continue
        
        print("📄 No more pages available")
        return False
        
    except Exception as e:
        print(f"⚠️ Error checking pagination: {e}")
        return False


def extract_match_data(soup):
    """
    Extract match data from the parsed HTML
    Returns list of dictionaries with match information
    """
    matches = []
    
    try:
        # Find the result list section
        result_section = soup.find("section", class_="result-list")
        if not result_section:
            print("❌ Could not find result-list section")
            return matches
        
        # Find all tournament blocks (dl elements)
        tournament_blocks = result_section.find_all("dl", class_="list")
        print(f"🏆 Found {len(tournament_blocks)} tournament blocks")
        
        for block in tournament_blocks:
            try:
                # Get tournament name from dt tag
                dt_tag = block.find("dt")
                tournament = dt_tag.get_text(strip=True) if dt_tag else "Unknown Tournament"
                # Remove any surrounding double quotes
                tournament = tournament.strip('"')
                print(f"🏆 Processing tournament: {tournament}")
                
                # Find all matches in this tournament (dd tags)
                matches_in_tournament = block.find_all("dd")
                print(f"⚽ Found {len(matches_in_tournament)} matches in {tournament}")
                
                for match_dd in matches_in_tournament:
                    try:
                        # Find the result-event ul
                        result_event = match_dd.find("ul", class_="result-event")
                        if not result_event:
                            continue
                        
                        # Extract home team, away team, and score
                        home_li = result_event.find("li", class_="home")
                        away_li = result_event.find("li", class_="away")
                        score_li = result_event.find("li", class_="score")
                        
                        if home_li and away_li and score_li:
                            home_team = home_li.get_text(strip=True)
                            away_team = away_li.get_text(strip=True)

                            # Extract score from score-com div - handle two different structures
                            score_div = score_li.find("div", class_="score-com")
                            if score_div:
                                score_detail = score_div.find("div", class_="score-detail")
                                
                                if score_detail:
                                    # Structure with halftime data - score is in score-com but outside score-detail
                                    # Get direct text content from score-com, excluding nested elements
                                    score_com_children = list(score_div.children)
                                    score_text = ""
                                    for child in score_com_children:
                                        if hasattr(child, 'name') and child.name:
                                            # Skip nested div elements
                                            continue
                                        else:
                                            # Get text nodes directly in score-com
                                            score_text += str(child).strip()
                                    
                                    # If no direct text found, try alternative extraction
                                    if not score_text or not any(c.isdigit() for c in score_text):
                                        full_text = score_div.get_text(separator='|', strip=True)
                                        detail_text = score_detail.get_text(strip=True)
                                        parts = full_text.split('|')
                                        # Look for the part that's not the halftime score
                                        for part in parts:
                                            if part.strip() != detail_text and ':' in part:
                                                score_text = part.strip()
                                                break
                                else:
                                    # Simple structure - score is directly in score-com
                                    score_text = score_div.get_text(strip=True)
                                
                                ft_goals = calculate_total_goals(score_text)
                                
                                match_data = {
                                    'tournament': tournament,
                                    'home_team': home_team,
                                    'away_team': away_team,
                                    'ft_goals': ft_goals
                                }
                                matches.append(match_data)
                                print(f"✅ Extracted: {home_team} vs {away_team} ({ft_goals} goals)")
                            
                    except Exception as e:
                        print(f"⚠️ Error processing individual match: {e}")
                        continue
                        
            except Exception as e:
                print(f"⚠️ Error processing tournament block: {e}")
                continue
    
    except Exception as e:
        print(f"❌ Error in extract_match_data: {e}")
    
    return matches


def scrape_sb_results(target_date):
    """
    Main scraping function for sb live results
    Args:
        target_date: Date string in format "05/09/2025"
    """
    url = "https://www.sportybet.com/ng/liveResult/"
    headers = get_random_headers()
    all_matches = []

    try:
        print(f"🚀 Starting scraper for date: {target_date}")
        
        # Set up headless Chrome
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-logging")
        chrome_options.add_argument("--log-level=3")
        chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
        chrome_options.add_argument("--window-size=1920,1080")
        
        # Initialize driver
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        
        print("🌐 Loading sb page...")
        driver.get(url)
        
        # Wait for initial page load
        time.sleep(random.uniform(3, 5))
        
        # Select the target date
        if not select_date(driver, target_date):
            print("❌ Failed to select target date")
            driver.quit()
            return []
        
        page_count = 1
        
        # Process all pages for the selected date
        while True:
            print(f"📄 Processing page {page_count}...")
            
            # Wait for content to load
            time.sleep(random.uniform(2, 4))
            
            # Get page source and parse
            page_source = driver.page_source
            
            # Clean the page source
            page_source = re.sub(r'/[^<>]*?\.wdm/[^<>]*?chromedriver[^<>]*?', '', page_source)
            page_source = re.sub(r'\[[^<>\[\]]*?chromedriver[^<>\[\]]*?\]', '', page_source)
            
            # Parse with BeautifulSoup
            try:
                soup = BeautifulSoup(page_source, 'html.parser')
            except Exception as e1:
                print(f"⚠️ html.parser failed: {e1}")
                try:
                    soup = BeautifulSoup(page_source, 'lxml')
                except Exception as e2:
                    print(f"❌ All parsers failed: {e2}")
                    break
            
            # Extract match data from current page
            page_matches = extract_match_data(soup)
            all_matches.extend(page_matches)
            print(f"📊 Extracted {len(page_matches)} matches from page {page_count}")
            
            # Check if there are more pages
            if not check_and_navigate_pagination(driver):
                break
                
            page_count += 1
            
            # Safety limit to prevent infinite loops
            if page_count > 50:  # Reasonable limit
                print("⚠️ Reached page limit, stopping pagination")
                break
        
        driver.quit()
        print(f"🎉 Scraping completed! Total matches extracted: {len(all_matches)}")
        return all_matches

    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        if 'driver' in locals():
            driver.quit()
        return []


def main():
    """
    Example usage of the scraper
    """
    # Example date - modify as needed
    target_date = "05/09/2025"  # Format: DD/MM/YYYY
    
    print(f"🔥 Starting sb scraper for date: {target_date}")
    
    # Run the scraper
    results = scrape_sb_results(target_date)
    
    if results:
        print(f"\n📈 Successfully extracted {len(results)} matches")
        
        # Save to CSV
        success, filename = save_to_csv(results)
        
        if success:
            print(f"✅ Data saved successfully to {filename}")
            
            # Display first few results as preview
            print("\n📋 Preview of extracted data:")
            for i, match in enumerate(results[:5]):  # Show first 5 matches
                print(f"{i+1}. {match['tournament']}: {match['home_team']} vs {match['away_team']} ({match['ft_goals']} goals)")
        else:
            print("❌ Failed to save data to CSV")
    else:
        print("❌ No data extracted")


if __name__ == "__main__":
    main()

🔥 Starting SportyBet scraper for date: 05/09/2025
🚀 Starting scraper for date: 05/09/2025
🌐 Loading SportyBet page...
🗓️ Attempting to select date: 05/09/2025
✅ Found dropdown with selector: .m-select-list
🔍 Found date option: '05/09/2025'
✅ Selecting date: 05/09/2025
📄 Processing page 1...
🏆 Found 28 tournament blocks
🏆 Processing tournament: International - World Cup Qualification UEFA
⚽ Found 10 matches in International - World Cup Qualification UEFA
✅ Extracted: Denmark vs Scotland (0 goals)
✅ Extracted: Faroe Islands vs Croatia (1 goals)
✅ Extracted: Greece vs Belarus (6 goals)
✅ Extracted: Iceland vs Azerbaijan (5 goals)
✅ Extracted: Italy vs Estonia (5 goals)
✅ Extracted: Moldova vs Israel (4 goals)
✅ Extracted: Montenegro vs Czechia (2 goals)
✅ Extracted: Slovenia vs Sweden (4 goals)
✅ Extracted: Switzerland vs Kosovo (4 goals)
✅ Extracted: Ukraine vs France (2 goals)
🏆 Processing tournament: International - World Cup Qualification, CONMEBOL
⚽ Found 5 matches in International -